In [1]:
suppressMessages(library(Seurat))
suppressMessages(library(ggplot2))
suppressMessages(library(patchwork))
suppressMessages(library(dplyr))
suppressMessages(library(CellChat))
suppressMessages(library(foreach))

In [2]:
in_dir <- "../../results/11_mac_endo/01_integrate_data"
out_dir <- "../../results/11_mac_endo/02_run_cellchat"

if(!dir.exists(out_dir)){
    dir.create(out_dir, recursive = TRUE)
}

In [3]:
seurat_object <- readRDS(glue::glue("{in_dir}/obj.rds"))

In [4]:
head(rownames(seurat_object))

[1] "A1BG"     "A1BG-AS1" "A2M"      "A2M-AS1"  "A2ML1"    "A4GALT"

In [5]:
cellchat <- createCellChat(object = seurat_object, group.by = "sub_clusters")

[1] "Create a CellChat object from a Seurat object"
The `data` slot in the default assay is used. The default assay is RNA 
The `meta.data` slot in the Seurat object is used as cell meta information 


Warning message in createCellChat(object = seurat_object, group.by = "sub_clusters"):
“The 'meta' data does not have a column named `samples`. We now add this column and all cells are assumed to belong to `sample1`! 
”


Set cell identities for the new CellChat object 
The cell groups used for CellChat analysis are  Antigen-presenting, Phagocytic, Pro-inflammatory, ArtEC, CapEC, EndoMT-1, EndoMT-2, LEC, VenEC 


In [6]:
CellChatDB <- CellChatDB.human

In [7]:
# use all CellChatDB except for "Non-protein Signaling" for cell-cell communication analysis
CellChatDB.use <- subsetDB(CellChatDB)

In [8]:
cellchat@DB <- CellChatDB.use

In [9]:
cellchat <- CellChat::subsetData(cellchat) # This step is necessary even if using the whole database

cellchat <- CellChat::identifyOverExpressedGenes(cellchat)
cellchat <- CellChat::identifyOverExpressedInteractions(cellchat, variable.both = F)

The number of highly variable ligand-receptor pairs used for signaling inference is 1277 


In [10]:
cellchat <- computeCommunProb(cellchat, type = "triMean")
cellchat <- CellChat::filterCommunication(cellchat, min.cells = 10)

triMean is used for calculating the average gene expression per cell group. 
[1] ">>> Run CellChat on sc/snRNA-seq data <<< [2024-12-12 15:05:35.868072]"
[1] ">>> CellChat inference is done. Parameter values are stored in `object@options$parameter` <<< [2024-12-12 15:08:21.577673]"


In [11]:
cellchat <- computeCommunProbPathway(cellchat)
cellchat <- aggregateNet(cellchat)

In [12]:
saveRDS(cellchat, glue::glue("{out_dir}/obj.rds"))